In [1]:
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
#필요한 패키지들 import하기
import pandas as pd
# import modin.pandas as pd
import numpy as np
from tqdm import tqdm
import os
import re
import gzip
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
path = '/data2/je_hong/kaist/kaist/physionet.org/files/mimiciv/2.0/icu/'

In [3]:
new_path = '/data2/je_hong/kaist/kaist/physionet.org/files/mimiciv/2.0/mimic_rev/'

In [4]:
hos_path = '/data2/je_hong/kaist/kaist/physionet.org/files/mimiciv/2.0/hosp/'

In [5]:
import os
os.listdir(path)

['icustays.csv.gz',
 'outputevents.csv.gz',
 'procedureevents.csv.gz',
 'ingredientevents.csv.gz',
 'index.html',
 'd_items.csv.gz',
 'datetimeevents.csv.gz',
 'chartevents.csv.gz',
 'inputevents.csv.gz']

In [6]:
with gzip.open(hos_path+'admissions.csv.gz') as f:
    admissions = pd.read_csv(f, chunksize=10)
    admissions = list(admissions)
    admissions = pd.concat(admissions)
print(admissions.shape)

(454324, 15)


In [7]:
feature_info = pd.read_csv("/data2/je_hong/kaist/kaist/physionet.org/files/mimiciv/2.0/mimic_rev/feature_info/feature_itemid.csv",  encoding='CP949')

In [8]:
with gzip.open(hos_path+'patients.csv.gz') as f:
    patients = pd.read_csv(f, chunksize=10)
    patients = list(patients)
    patients = pd.concat(patients)
print(patients.shape)

(315460, 6)


In [170]:
# with gzip.open(hos_path+'transfers.csv.gz') as f:
#     transfers = pd.read_csv(f, chunksize=10)
#     transfers = list(transfers)
#     transfers = pd.concat(transfers)
# print(transfers.shape)


KeyboardInterrupt: 

In [ ]:
# careunits 

In [9]:
with gzip.open(path+'icustays.csv.gz') as f:
    icustays = pd.read_csv(f, chunksize=10)
    icustays = list(icustays)
    icustays = pd.concat(icustays)
print(icustays.shape)

(76943, 8)


In [10]:
new_path

'/data2/je_hong/kaist/kaist/physionet.org/files/mimiciv/2.0/mimic_rev/'

In [98]:
# with gzip.open(new_path+'drop_2_chartevents.gz') as f:
#     drop_chartevents = pd.read_csv(f, chunksize=10**2)
#     drop_chartevents = list(drop_chartevents)
#     drop_chartevents = pd.concat(drop_chartevents)
# print(drop_chartevents.shape)

In [11]:
features = feature_info['feature'].unique()

In [12]:
icustays

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535
2,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032
3,10001217,27703517,34592300,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113
4,10001725,25563031,31205490,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588
...,...,...,...,...,...,...,...,...
76938,19999442,26785317,32336619,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370
76939,19999625,25304202,31070865,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741
76940,19999828,25744818,36075953,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995
76941,19999840,21033226,38978960,Trauma SICU (TSICU),Surgical Intensive Care Unit (SICU),2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766


In [13]:
admissions.columns

Index(['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime',
       'admission_type', 'admission_location', 'discharge_location',
       'insurance', 'language', 'marital_status', 'race', 'edregtime',
       'edouttime', 'hospital_expire_flag'],
      dtype='object')

In [14]:
adm_stay = pd.merge(left=admissions, right=icustays, how='inner', on='hadm_id')

In [15]:
adm_stay['deathtime'].value_counts()

2127-11-04 03:25:00    6
2166-07-06 23:45:00    6
2199-10-25 14:30:00    6
2128-11-14 00:01:00    6
2194-05-11 06:30:00    6
                      ..
2120-02-22 17:00:00    1
2129-08-23 01:30:00    1
2163-01-08 10:00:00    1
2150-10-04 21:29:00    1
2141-08-23 07:59:00    1
Name: deathtime, Length: 7337, dtype: int64

In [16]:
adm_stay.columns

Index(['subject_id_x', 'hadm_id', 'admittime', 'dischtime', 'deathtime',
       'admission_type', 'admission_location', 'discharge_location',
       'insurance', 'language', 'marital_status', 'race', 'edregtime',
       'edouttime', 'hospital_expire_flag', 'subject_id_y', 'stay_id',
       'first_careunit', 'last_careunit', 'intime', 'outtime', 'los'],
      dtype='object')

In [119]:
# die_pat = admissions[admissions['hospital_expire_flag'] == 1]

In [17]:
## data type 변경
adm_stay['deathtime'] = pd.to_datetime(adm_stay['deathtime'])
adm_stay['intime'] = pd.to_datetime(adm_stay['intime'])
adm_stay['outtime'] = pd.to_datetime(adm_stay['outtime'])

In [18]:
adm_stay.columns

Index(['subject_id_x', 'hadm_id', 'admittime', 'dischtime', 'deathtime',
       'admission_type', 'admission_location', 'discharge_location',
       'insurance', 'language', 'marital_status', 'race', 'edregtime',
       'edouttime', 'hospital_expire_flag', 'subject_id_y', 'stay_id',
       'first_careunit', 'last_careunit', 'intime', 'outtime', 'los'],
      dtype='object')

## str -> datetime

In [97]:
from datetime import datetime

NOT_CONVERTED = 'NOT_CONVERTED'


def str2datetime(s):
    def _convert(_s, _dformat):
        try:
            converted_dt = datetime.strptime(_s, _dformat)
        except Exception:
            return NOT_CONVERTED

        return converted_dt

    if isinstance(s, datetime):
        return s

    dformats = [
        '%Y-%m-%d %p %I:%M:%S',
        '%Y-%m-%d %H:%M:%S.%f',
        '%Y-%m-%d %H:%M:%S',
        '%Y-%m-%d %H:%M',
        '%y-%m-%d %H:%M',
        '%Y-%m-%d',
        '%Y%m%d',
        '%Y-%m-%d +%H:%M',
        '%Y%m%d %H%M',
        '%Y%m%d%H%M%S',
        ]

    s = s.split('.')[0]
    for dformat in dformats:
        dt = _convert(s, dformat)
        if dt != NOT_CONVERTED:
            return dt

    return NOT_CONVERTED

In [39]:
duplicated_hadm_id = list(set(adm_stay[adm_stay.duplicated(['hadm_id'], keep=False)]['hadm_id']))

In [71]:
adm_stay = adm_stay.sort_values(by=['admittime'], axis=0)

In [ ]:
## hadm_id 기준으로 돌리는게 맞을 것 같다.

In [57]:
new_path

'/data2/je_hong/kaist/kaist/physionet.org/files/mimiciv/2.0/mimic_rev/'

In [109]:
adm_stay.columns

Index(['subject_id_x', 'hadm_id', 'admittime', 'dischtime', 'deathtime',
       'admission_type', 'admission_location', 'discharge_location',
       'insurance', 'language', 'marital_status', 'race', 'edregtime',
       'edouttime', 'hospital_expire_flag', 'subject_id_y', 'stay_id',
       'first_careunit', 'last_careunit', 'intime', 'outtime', 'los',
       'death_in_icu'],
      dtype='object')

## labeling 기준 되는 feature nan 값 제외

In [114]:
adm_stay = adm_stay.dropna(subset=['hadm_id', 'deathtime', 'intime', 'outtime', 'stay_id', 'hospital_expire_flag'])

In [152]:
adm_stay[adm_stay.duplicated(['subject_id_y'], keep=False)]['subject_id_y']

25139    13263248
25138    13263248
25140    13263248
66634    18648460
66633    18648460
           ...   
42571    15525793
54570    17083786
54571    17083786
20507    12648465
20508    12648465
Name: subject_id_y, Length: 2568, dtype: int64

## 의문점: hadm_id 가 중복된 값 == stay_id 가 여러개 라는 뜻인데, 어차피 1개 밖에 못쓰는데,,  ** stay_id 무조건 첫 값을 쓰라고 했다..? 음..

In [149]:
adm_stay[adm_stay.duplicated(['hadm_id'], keep=False)]

,subject_id_x,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,...,edouttime,hospital_expire_flag,subject_id_y,stay_id,first_careunit,last_careunit,intime,outtime,los,death_in_icu
25139,13263248,22811618,2110-02-10 12:37:00,2110-03-07 01:37:00,2110-03-07 01:37:00,URGENT,TRANSFER FROM HOSPITAL,DIED,Medicare,ENGLISH,...,2110-02-10 14:06:00,1,13263248,35243308,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2110-02-10 14:06:00,2110-02-13 21:26:39,3.306007,NaN
25138,13263248,22811618,2110-02-10 12:37:00,2110-03-07 01:37:00,2110-03-07 01:37:00,URGENT,TRANSFER FROM HOSPITAL,DIED,Medicare,ENGLISH,...,2110-02-10 14:06:00,1,13263248,34092954,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2110-02-24 17:26:36,2110-02-26 17:19:24,1.995000,NaN
25140,13263248,22811618,2110-02-10 12:37:00,2110-03-07 01:37:00,2110-03-07 01:37:00,URGENT,TRANSFER FROM HOSPITAL,DIED,Medicare,ENGLISH,...,2110-02-10 14:06:00,1,13263248,39247539,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2110-02-26 21:35:20,2110-03-07 04:09:42,8.273866,NaN
66634,18648460,23908012,2110-03-28 18:22:00,2110-04-03 00:00:00,2110-04-03 00:00:00,OBSERVATION ADMIT,TRANSFER FROM HOSPITAL,DIED,Other,?,...,NaN,1,18648460,38911590,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2110-03-29 08:39:45,2110-03-29 20:48:26,0.506030,NaN
66633,18648460,23908012,2110-03-28 18:22:00,2110-04-03 00:00:00,2110-04-03 00:00:00,OBSERVATION ADMIT,TRANSFER FROM HOSPITAL,DIED,Other,?,...,NaN,1,18648460,34111818,Trauma SICU (TSICU),Coronary Care Unit (CCU),2110-03-31 08:40:44,2110-04-03 06:03:14,2.890625,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42571,15525793,22462046,2206-09-11 17:02:00,2206-09-19 11:38:00,2206-09-19 11:38:00,EW EMER.,EMERGENCY ROOM,DIED,Medicaid,ENGLISH,...,2206-09-11 22:34:00,1,15525793,36826483,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2206-09-17 00:28:38,2206-09-19 14:32:33,2.586053,NaN
54570,17083786,21830548,2209-11-11 04:12:00,2209-11-22 05:15:00,2209-11-22 05:15:00,EW EMER.,PHYSICIAN REFERRAL,DIED,Medicare,ENGLISH,...,2209-11-11 07:50:00,1,17083786,32909332,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2209-11-11 06:20:58,2209-11-12 11:59:08,1.234838,NaN
54571,17083786,21830548,2209-11-11 04:12:00,2209-11-22 05:15:00,2209-11-22 05:15:00,EW EMER.,PHYSICIAN REFERRAL,DIED,Medicare,ENGLISH,...,2209-11-11 07:50:00,1,17083786,39363395,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2209-11-13 17:18:25,2209-11-16 18:49:52,3.063507,NaN
20507,12648465,22241351,2210-07-20 03:56:00,2210-09-16 02:00:00,2210-09-16 02:00:00,EW EMER.,PHYSICIAN REFERRAL,DIED,Other,ENGLISH,...,2210-07-20 07:49:00,1,12648465,35859492,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2210-09-06 23:54:33,2210-09-16 07:58:16,9.335914,NaN


In [115]:
adm_stay[adm_stay['hospital_expire_flag'] == 1]['deathtime'].isnull().sum()

0

In [116]:
adm_stay.to_csv(new_path+'new_csv/adm_stay.csv')

In [154]:
HADMS = adm_stay['hadm_id'].tolist()

In [180]:
#Function of making profiles
def make_profile(profile_dic,hadm_id, intime, outtime, dod):
    profile_list = profile_dic.get(hadm_id, list())
    profile_list.append({'INTIME': intime, 'OUTTIME':outtime, 'IS_DEAD': is_dead, 'DOD': dod})
    profile_dic[hadm_id] = profile_list
    return profile_dic

In [191]:
len(set(HADMS))

7338

## labeling 

#     hadm_id, intime, outtime, hospital_expire_flag, deathtime

## hadm_id 중복값 제외해주고 코딩

In [187]:
profile_dic = dict()
for hadm_id in list(set(HADMS)):
    intime = (adm_stay.loc[adm_stay['hadm_id'] == hadm_id]['intime'].values[0])
    outtime = (adm_stay.loc[adm_stay['hadm_id'] == hadm_id]['outtime'].values[0])
    is_dead = int(adm_stay.loc[adm_stay['hadm_id'] == hadm_id]['hospital_expire_flag'].values[0])
    dod = (adm_stay.loc[adm_stay['hadm_id'] == hadm_id]['deathtime'].values[0])
    if is_dead == 1:
        if outtime >= dod:
            is_dead = 1
        else:
            is_dead = 0
            dod = None
    else:
        pass
    profile_dic = make_profile(profile_dic, hadm_id, intime, outtime, dod)


In [194]:
profile_dic = dict()
for hadm_id in list(set(HADMS)):
    print(adm_stay.loc[adm_stay['hadm_id'] == hadm_id]['intime'].values[0])
#     outtime = (adm_stay.loc[adm_stay['hadm_id'] == hadm_id]['outtime'].values[0])
#     is_dead = int(adm_stay.loc[adm_stay['hadm_id'] == hadm_id]['hospital_expire_flag'].values[0])
#     dod = (adm_stay.loc[adm_stay['hadm_id'] == hadm_id]['deathtime'].values[0])
#     if is_dead == 1:
#         if outtime >= dod:
#             is_dead = 1
#         else:
#             is_dead = 0
#             dod = None
#     else:
#         pass
#     profile_dic = make_profile(profile_dic, hadm_id, intime, outtime, dod)


2185-07-04T22:43:50.000000000
2144-08-03T20:00:14.000000000
2180-04-18T18:56:16.000000000
2150-08-27T09:02:41.000000000
2182-06-07T18:09:00.000000000
2119-06-14T18:53:05.000000000
2195-01-30T17:22:00.000000000
2146-07-23T00:44:00.000000000
2183-09-02T22:39:00.000000000
2118-07-10T14:18:23.000000000
2169-09-02T04:45:00.000000000
2181-06-16T16:13:00.000000000
2157-05-03T00:10:00.000000000
2161-04-10T16:19:00.000000000
2163-01-18T20:11:12.000000000
2151-03-29T23:58:00.000000000
2155-10-24T16:56:15.000000000
2113-08-15T22:06:00.000000000
2183-09-03T09:58:13.000000000
2147-08-12T03:27:47.000000000
2167-09-08T21:58:18.000000000
2116-12-05T20:15:00.000000000
2184-09-01T13:40:48.000000000
2144-02-05T22:36:27.000000000
2176-06-25T19:52:00.000000000
2157-07-09T05:19:00.000000000
2129-02-20T11:19:41.000000000
2179-09-28T13:01:45.000000000
2116-10-12T21:48:10.000000000
2123-01-04T18:38:00.000000000
2129-03-25T22:37:17.000000000
2155-06-28T21:52:25.000000000
2131-06-17T02:27:24.000000000
2120-05-08

In [188]:
len(profile_dic)

7338

In [197]:
## labeling 잘 되었는지 체크
for i in profile_dic:
    if (profile_dic[i]['IS_DEAD'] == 1):
        print(profile_dic[i]['DOD'], profile_dic[i]['OUTTIME'])

In [ ]:
## 아래는 처음 짠 코드

In [91]:
## 환자 사망? -> 사망 시간? 
#  stay_id 안겹침
profile_dic = dict()
for idx, row in adm_stay.iterrows():
    sub_id = int(row['subject_id_y']) 
    hadm_id = int(row['hadm_id'])
    if hadm_id in duplicated_hadm_id:
        print(adm_stay.loc[adm_stay['hadm_id'] == hadm_id]['stay_id'].tolist()[0])
    intime = str2datetime(row['intime'])
    outtime = str2datetime(row['outtime'])
    deathtime = str2datetime(row['deathtime'])
    hospital_expire_flag = row['hospital_expire_flag']
#         sub_list.append("{};{};{};{};{};{}".format(sub_id, hadm_id, intime, outtime, hospital_expire_flag, deathtime))
#     sub_id, hadm_id, intime, outtime, hospital_expire_flag
#     sub_id hadm_id, intime, outtime, hospital_expire_flag

34353159
34353159
37409541
37409541
38128253
38128253
39041457
39041457
35243308
35243308
35243308
33592773
33592773
39444389
39444389
39577681
39577681
39577681
38911590
38911590
36352845
36352845
39934754
39934754
31805668
31805668
30623123
30623123
37345888
37345888
39065822
39065822
36497526
36497526
35520979
35520979
34121177
34121177
36149370
36149370
34314825
34314825
33990314
33990314
34806666
34806666
38049303
38049303
38049303
36763879
36763879
36763879
33388628
33388628
31060997
31060997
31060997
35423873
35423873
38765635
38765635
30137702
30137702
30853100
30853100
30853100
34159119
34159119
39885765
39885765
38774055
38774055
31688642
31688642
39919566
39919566
39919566
37449164
37449164
30305507
30305507
36086238
36086238
34325526
34325526
30985329
30985329
37403874
37403874
37403874
36746901
36746901
32246373
32246373
32874998
32874998
30636501
30636501
31122387
31122387
31122387
31651591
31651591
32878666
32878666
30810688
30810688
35114990
35114990
35114990
30686999
3

KeyboardInterrupt: 

In [27]:
len(sub_list)

76943

### 만약에 hadm_id 가 중복 & stay_id가 여러개가 있으면 앞시간대를 무조건 고려.... *****

In [28]:
icustays

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535
2,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032
3,10001217,27703517,34592300,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113
4,10001725,25563031,31205490,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588
...,...,...,...,...,...,...,...,...
76938,19999442,26785317,32336619,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370
76939,19999625,25304202,31070865,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741
76940,19999828,25744818,36075953,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995
76941,19999840,21033226,38978960,Trauma SICU (TSICU),Surgical Intensive Care Unit (SICU),2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766


In [29]:
adm_stay['death_in_icu'] = pd.Series()

In [30]:
for i in sub_list:
    if (i.split(";")[-2]) == 0 or (i.split(";")[-2]) == 1:
        pass
    else:
        (i.split(";")[-2])

In [31]:
# icu내에서 죽은 환자인지
for i in sub_list:
    # 죽은 환자 = 1
    # 안죽고 퇴실한 환자 = ?
    if (i.split(";")[-2]) == '0':
        print(i.split(';')[-1])

NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT


## deathtime < outtime  = 1 & else = 0 

In [179]:
adm_stay.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76943 entries, 0 to 76942
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   subject_id_x          76943 non-null  int64         
 1   hadm_id               76943 non-null  int64         
 2   admittime             76943 non-null  object        
 3   dischtime             76943 non-null  object        
 4   deathtime             8785 non-null   datetime64[ns]
 5   admission_type        76943 non-null  object        
 6   admission_location    76943 non-null  object        
 7   discharge_location    76271 non-null  object        
 8   insurance             76943 non-null  object        
 9   language              76943 non-null  object        
 10  marital_status        71562 non-null  object        
 11  race                  76943 non-null  object        
 12  edregtime             50728 non-null  object        
 13  edouttime       

In [180]:
adm_stay['hadm_id']

0        29079034
1        26913865
2        24597018
3        27703517
4        25563031
           ...   
76938    26785317
76939    25304202
76940    25744818
76941    21033226
76942    23865745
Name: hadm_id, Length: 76943, dtype: int64

In [ ]:
# icu내에서 죽은 환자인지
for i in sub_list:
    # 죽은 환자 = 1
    # 안죽고 퇴실한 환자 = ?
    if (i.split(";")[-2]) == '1':
        # 만약 outtime 이 deathtime 보다 크면, worse로 간주
        if (i.split(';')[-3]) >= (i.split(';')[-1]):
            print(adm_stay.loc[adm_stay['hadm_id'] == (i.split(';')[1])])
            
#         else:
#             adm_stay.loc[adm_stay['hadm_id'] == int(i.split(';')[1])]['death_in_icu'] = 1
#         else:
#             adm_stay.loc[adm_stay['hadm_id'] == int(i.split(';')[1])]['death_in_icu'] = 0
#     else:
#         adm_stay.loc[adm_stay['hadm_id'] == int(i.split(';')[1])]['death_in_icu'] = 0

In [137]:
str2datetime(sub_list[0].split(';')[2])

datetime.datetime(2180, 7, 23, 14, 0)

In [138]:
sub_list[0].split(';')[2]

'2180-07-23 14:00:00'

In [131]:
adm_stay

,subject_id_x,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,...,edouttime,hospital_expire_flag,subject_id_y,stay_id,first_careunit,last_careunit,intime,outtime,los,death_in_icu
0,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,...,2180-07-23 14:00:00,0,10000032,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,NaN
1,10000980,26913865,2189-06-27 07:38:00,2189-07-03 03:00:00,NaT,EW EMER.,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,ENGLISH,...,2189-06-27 08:42:00,0,10000980,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535,NaN
2,10001217,24597018,2157-11-18 22:56:00,2157-11-25 18:00:00,NaT,EW EMER.,EMERGENCY ROOM,HOME HEALTH CARE,Other,?,...,2157-11-19 01:24:00,0,10001217,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,NaN
3,10001217,27703517,2157-12-18 16:58:00,2157-12-24 14:55:00,NaT,DIRECT EMER.,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,?,...,NaN,0,10001217,34592300,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113,NaN
4,10001725,25563031,2110-04-11 15:08:00,2110-04-14 15:00:00,NaT,EW EMER.,PACU,HOME,Other,ENGLISH,...,NaN,0,10001725,31205490,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76938,19999442,26785317,2148-11-19 10:00:00,2148-12-04 16:25:00,NaT,ELECTIVE,PHYSICIAN REFERRAL,REHAB,Medicaid,ENGLISH,...,NaN,0,19999442,32336619,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370,NaN
76939,19999625,25304202,2139-10-10 18:06:00,2139-10-16 03:30:00,NaT,EW EMER.,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,?,...,2139-10-10 19:18:00,0,19999625,31070865,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741,NaN
76940,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaT,EW EMER.,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,...,2149-01-08 18:12:00,0,19999828,36075953,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995,NaN
76941,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,EW EMER.,EMERGENCY ROOM,DIED,Other,ENGLISH,...,2164-09-10 14:46:00,1,19999840,38978960,Trauma SICU (TSICU),Surgical Intensive Care Unit (SICU),2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766,NaN


In [133]:
adm_stay['death_in_icu'].value_counts()

Series([], Name: death_in_icu, dtype: int64)